In [1]:
import pandas as pd

In [2]:
df = pd.read_stata('Add Health, Public Use In-Home Data, Wave IV.DTA')

In [3]:
df.shape

(5114, 920)

In [4]:
df.columns

Index([u'aid', u'IMONTH4', u'IDAY4', u'IYEAR4', u'BIO_SEX4', u'VERSION4',
       u'BREAK_Q', u'PRYEAR4', u'PRETEST4', u'PRISON4',
       ...
       u'H4EO5C', u'H4EO5D', u'H4EO5E', u'H4EO5F', u'H4EO5G', u'H4EO5H',
       u'H4EO5I', u'H4EO5J', u'H4EO6', u'H4EO7'],
      dtype='object', length=920)

In [5]:
df['sex'] = df.BIO_SEX4
df.sex.value_counts()

Female    2761
Male      2353
Name: sex, dtype: int64

In [6]:
df['yob'] = df.H4OD1Y
df.yob.value_counts()

1979    914
1978    887
1980    884
1977    875
1981    724
1982    464
1976    307
1975     45
1983      7
1974      7
Name: yob, dtype: int64

In [7]:
df['race'] = df.H4IR4
df.race.value_counts()

White                               3671
Black or African American           1240
Asian or Pacific Islander            157
American Indian or Alaska Native      41
Name: race, dtype: int64

In [8]:
df['friends'] = df.H4WS4
df.friends.value_counts()

Three to five friends    2319
One or two friends       1121
Six to nine friends       861
10 or more friends        620
None                      145
Not asked on pretest       46
Don't know                  1
Refused                     1
Name: friends, dtype: int64

In [9]:
df['religion'] = df.H4RE1
df.religion.value_counts()

Protestant (such as Assemblies of God, Baptist, Lutheran, Methodist, Presbyterian, etc.)    1675
Other Christian                                                                             1128
None/atheist/agnostic                                                                        962
Catholic                                                                                     925
Other                                                                                        330
Jewish                                                                                        34
Buddhist                                                                                      24
Muslim                                                                                        14
Refused                                                                                       11
Don't know                                                                                     7
Hindu                         

In [10]:
df['NONE'] = df.religion == 'None/atheist/agnostic'
df.NONE.value_counts()

False    4152
True      962
Name: NONE, dtype: int64

In [11]:
df['attend'] = df.H4RE7
df.attend.value_counts()

A few times                   1626
Never                         1509
Two or three times a month     649
Once a week                    575
Once a month                   404
More than once a week          349
Don't know                       2
Name: attend, dtype: int64

In [12]:
df['NOATTEND'] = df.attend == 'Never'
df.NOATTEND.value_counts()

False    3605
True     1509
Name: NOATTEND, dtype: int64

In [13]:
df['relimp'] = df.H4RE9
df.relimp.value_counts()

Very important                       2255
Somewhat important                   1438
Not important                         764
More important than anything else     642
Refused                                 9
Don't know                              6
Name: relimp, dtype: int64

In [14]:
df['relturn'] = df.H4RE11
df.relturn.value_counts()

Very often    1359
Never         1066
Often         1020
Sometimes      994
Seldom         660
Refused         12
Don't know       3
Name: relturn, dtype: int64

In [15]:
df['opioid'] = df.H4TO64D
df.opioid.value_counts()

Legitimate skip    4223
Yes                 721
No                  160
Don't know           10
Name: opioid, dtype: int64

In [16]:
df['OPIOID'] = (df.opioid == 'Yes').astype(int)
df.OPIOID.value_counts()

0    4393
1     721
Name: OPIOID, dtype: int64

In [17]:
df['everpot'] = df.H4TO65B
df.everpot.value_counts()

Yes           2778
No            2300
Refused         26
Don't know      10
Name: everpot, dtype: int64

In [18]:
df['evermeth'] = df.H4TO65D
df.evermeth.value_counts()

No            4629
Yes            459
Refused         19
Don't know       7
Name: evermeth, dtype: int64

In [19]:
import statsmodels.formula.api as smf


In [20]:
formula = 'OPIOID ~ NONE + sex + yob + C(race)'
model1 = smf.logit(formula=formula, data=df).fit()
model1.summary()

Optimization terminated successfully.
         Current function value: 0.384921
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 OPIOID   No. Observations:                 5109
Model:                          Logit   Df Residuals:                     5102
Method:                           MLE   Df Model:                            6
Date:                Sun, 29 Oct 2017   Pseudo R-squ.:                 0.05424
Time:                        16:26:45   Log-Likelihood:                -1966.6
converged:                       True   LL-Null:                       -2079.4
                                        LLR p-value:                 6.727e-46
===============================================================================================================
                                                  coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
Intercept                                    -177.9935     46.146     -3.857      0.000    -268.438     -87.548
NONE[T.True]                                    0.6083      0.092      6.594      0.000       0.427       0.789
sex[T.Female]                                  -0.4567      0.083     -5.508      0.000      -0.619      -0.294
C(race)[T.Black or African American]           -1.2267      0.135     -9.086      0.000      -1.491      -0.962
C(race)[T.American Indian or Alaska Native]    -0.6886      0.533     -1.292      0.196      -1.733       0.356
C(race)[T.Asian or Pacific Islander]           -0.6940      0.277     -2.505      0.012      -1.237      -0.151
yob                                             0.0892      0.023      3.825      0.000       0.043       0.135
===============================================================================================================
"""

In [21]:
new = pd.DataFrame(dict(NONE=False, sex='Male', race='White', yob=1980), index=[0])
model1.predict(new)

0    0.195919
dtype: float64

In [22]:
new = pd.DataFrame(dict(NONE=True, sex='Male', race='White', yob=1980), index=[0])
model1.predict(new)

0    0.309228
dtype: float64

In [23]:
formula = 'OPIOID ~ NOATTEND + sex + yob + C(race)'
model2 = smf.logit(formula=formula, data=df).fit()
model2.summary()

Optimization terminated successfully.
         Current function value: 0.382800
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 OPIOID   No. Observations:                 5109
Model:                          Logit   Df Residuals:                     5102
Method:                           MLE   Df Model:                            6
Date:                Sun, 29 Oct 2017   Pseudo R-squ.:                 0.05945
Time:                        16:26:45   Log-Likelihood:                -1955.7
converged:                       True   LL-Null:                       -2079.4
                                        LLR p-value:                 1.585e-50
===============================================================================================================
                                                  coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
Intercept                                    -179.2643     46.179     -3.882      0.000    -269.773     -88.756
NOATTEND[T.True]                                0.6747      0.084      8.022      0.000       0.510       0.840
sex[T.Female]                                  -0.4359      0.083     -5.240      0.000      -0.599      -0.273
C(race)[T.Black or African American]           -1.1845      0.135     -8.745      0.000      -1.450      -0.919
C(race)[T.American Indian or Alaska Native]    -0.6151      0.532     -1.157      0.247      -1.657       0.427
C(race)[T.Asian or Pacific Islander]           -0.6677      0.277     -2.407      0.016      -1.211      -0.124
yob                                             0.0898      0.023      3.847      0.000       0.044       0.135
===============================================================================================================
"""

In [24]:
new = pd.DataFrame(dict(NOATTEND=False, sex='Male', race='White', yob=1980), index=[0])
model2.predict(new)

0    0.177728
dtype: float64

In [25]:
new = pd.DataFrame(dict(NOATTEND=True, sex='Male', race='White', yob=1980), index=[0])
model2.predict(new)

0    0.297949
dtype: float64